In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.models import load_model
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import time

# Training Data

In [14]:
train_data = pd.read_csv("csv_files/train.csv")
train_data

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
0,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,150,1.20,45.78
1,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,160,1.12,39.00
2,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,170,1.06,32.78
3,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,180,1.00,27.17
4,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,190,0.95,22.18
...,...,...,...,...,...,...,...,...,...,...
1748995,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,200,0.86,25.49
1748996,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,210,0.82,22.21
1748997,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,220,0.78,19.28
1748998,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,230,0.75,16.67


In [15]:
train_data.describe()

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
count,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06
mean,2.770000e-01,1.710900e+00,3.634333e-01,9.103333e-02,-5.388667e-01,1.000000e+00,1.795318e+02,1.960000e+02,9.384750e-01,2.772866e+01
std,1.372941e-01,7.905263e-01,2.244740e-01,3.526943e-02,2.017475e-01,4.316752e-01,1.251597e+01,3.039738e+01,1.608779e-01,1.714504e+01
min,1.000000e-02,3.000000e-02,1.000000e-02,3.000000e-02,-9.000000e-01,0.000000e+00,1.500100e+02,1.500000e+02,6.000000e-01,-0.000000e+00
25%,1.600000e-01,1.097500e+00,1.700000e-01,6.000000e-02,-7.100000e-01,6.900000e-01,1.704700e+02,1.700000e+02,8.100000e-01,1.399000e+01
50%,2.800000e-01,1.765000e+00,3.400000e-01,9.000000e-02,-5.400000e-01,1.000000e+00,1.796900e+02,1.950000e+02,9.200000e-01,2.619000e+01
75%,4.000000e-01,2.392500e+00,5.525000e-01,1.200000e-01,-3.700000e-01,1.310000e+00,1.885400e+02,2.200000e+02,1.060000e+00,3.973000e+01
max,5.000000e-01,3.000000e+00,7.900000e-01,1.500000e-01,-2.000000e-01,2.000000e+00,2.099700e+02,2.500000e+02,1.400000e+00,1.006500e+02


In [16]:
# Obtain features and label -> X, Y
X, Y = train_data.drop(["moneyness", "price"], axis=1), train_data["price"]

In [17]:
# Normalize the data -> X_normalized, Y_normalized
## Normalize the model parameters in a domain
def custom_min_max_normalization(x, xmin, xmax):
    return (2 * x - (xmax + xmin)) / (xmax - xmin)

model_parameters = X.drop(["tau", "stockPrice", "strike"], axis=1)
option_properties = X.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

min_vals = pd.Series({ 
    'varsigma': 0.01,
    'kappa': 0,
    'v0': 0.03,
    'delta': 0.01,
    'rho': -0.9
})

max_vals = pd.Series({
    'varsigma': 0.5,
    'kappa': 3.0,
    'v0': 0.15,
    'delta': 0.8,
    'rho': -0.2
})

normalized_model_parameters = pd.DataFrame()
for column in model_parameters.columns:
    normalized_model_parameters[column] = custom_min_max_normalization(
        model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

scaler = MinMaxScaler() 
normalized_option_properties = scaler.fit_transform(option_properties)
normalized_option_properties = pd.DataFrame(normalized_option_properties, columns=option_properties.columns)

X_normalized = pd.concat([normalized_model_parameters, normalized_option_properties], axis=1).values

Y_normalized = scaler.fit_transform(Y.values.reshape(-1, 1))
Y_normalized = pd.Series(Y_normalized.flatten(), name=Y.name).values

# Test Data

In [29]:
test = pd.read_csv("csv_files/outOfSample.csv")
test = round(test,2)

In [30]:
test.describe()

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
count,42400.000000,42400.00000,42400.000000,42400.00000,42400.000000,42400.000000,42400.000000,42400.000000,42400.000000,42400.000000
mean,0.300600,1.72840,0.346600,0.09240,-0.525600,1.100000,176.103230,190.000000,0.939944,30.159162
std,0.130192,0.76819,0.225335,0.03592,0.194786,0.422904,10.120805,22.360943,0.124657,15.160794
min,0.040000,0.10000,0.010000,0.03000,-0.870000,0.250000,155.110000,160.000000,0.700000,0.010000
25%,0.210000,1.23000,0.150000,0.06000,-0.700000,0.805000,168.540000,175.000000,0.840000,18.660000
50%,0.305000,1.80000,0.310000,0.09000,-0.520000,1.100000,175.160000,190.000000,0.930000,29.560000
75%,0.420000,2.28000,0.520000,0.12000,-0.350000,1.395000,183.490000,205.000000,1.040000,40.860000
max,0.490000,2.99000,0.780000,0.15000,-0.210000,1.950000,199.870000,220.000000,1.250000,81.000000


In [31]:
TestX, TestY = test.drop(["moneyness","price"], axis=1), test["price"]

In [32]:
# Normalize TestX
test_model_parameters = TestX.drop(["tau", "stockPrice", "strike"], axis=1)
test_option_properties = TestX.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

normalized_test_model_parameters = pd.DataFrame()
for column in test_model_parameters.columns:
    normalized_test_model_parameters[column] = custom_min_max_normalization(
        test_model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

# Fit the scaler with option_properties from the training data
scaler_option_properties = MinMaxScaler() 
scaler_option_properties.fit(option_properties)

# Transform the option_properties from the test data
normalized_test_option_properties = scaler_option_properties.transform(test_option_properties)
normalized_test_option_properties = pd.DataFrame(normalized_test_option_properties, columns=test_option_properties.columns)

TestX_normalized = pd.concat([normalized_test_model_parameters, normalized_test_option_properties], axis=1).values

# Now the scaler for Y
scaler_Y = MinMaxScaler()
scaler_Y.fit(Y.values.reshape(-1, 1))

# Normalize TestY
TestY_normalized = scaler_Y.transform(TestY.values.reshape(-1, 1))  # Use transform, not fit_transform
TestY_normalized = pd.Series(TestY_normalized.flatten(), name=TestY.name).values

# Best FNN

## 4 layers, nodes = 64, epochs = 50

In [ ]:
# Data preprossesing -> X_train, X_test, X_val, Y_train, Y_test, Y_val
# Train - 80%, Test - 10%, Validation - 10%
num_samples = len(X)
train_size = int(0.8 * num_samples)
valid_size = int(0.1 * num_samples)
test_size = int(0.1 * num_samples)

# Split the data into train, validation, and test sets
X_train = X_normalized[:train_size]
Y_train = Y_normalized[:train_size]

X_val = X_normalized[train_size:train_size + valid_size]
Y_val = Y_normalized[train_size:train_size + valid_size]

X_test = X_normalized[train_size + valid_size:]
Y_test = Y_normalized[train_size + valid_size:]

def FNN():
    FNN = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(X_train.shape[1],), activation=tf.nn.relu),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation=tf.nn.relu)  
    ])
    return FNN

# Create the model
FNN = FNN()

# Compile the model
FNN.compile(optimizer='adam', loss='mse')

# Define early stopping with more patience
early_stopping = EarlyStopping(monitor='val_loss', patience=7)

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch != 0 and epoch % 20 == 0: 
        return lr * 0.5
    else:
        return lr
lr_scheduler = LearningRateScheduler(scheduler)

# Display the model summary
FNN.summary()

# Train the model
history = FNN.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    epochs=50, batch_size=32, verbose=True, callbacks=[early_stopping, lr_scheduler])

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,121 (51.25 KB)

 Trainable params: 13,121 (51.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 3.7172e-04 - val_loss: 1.2695e-05 - learning_rate: 0.0010
Epoch 2/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 135s 3ms/step - loss: 6.8915e-06 - val_loss: 6.6112e-06 - learning_rate: 0.0010
Epoch 3/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 118s 3ms/step - loss: 4.6300e-06 - val_loss: 6.7760e-06 - learning_rate: 0.0010
Epoch 4/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 120s 3ms/step - loss: 3.5974e-06 - val_loss: 7.5682e-06 - learning_rate: 0.0010
Epoch 5/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 148s 3ms/step - loss: 3.0764e-06 - val_loss: 3.6437e-06 - learning_rate: 0.0010
Epoch 6/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 151s 3ms/step - loss: 2.7527e-06 - val_loss: 3.6566e-06 - learning_rate: 0.0010
Epoch 7/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 129s 3ms/step - loss: 2.4704e-06 - val_loss: 5.2879e-06 - learning_rate: 0.0010
Epoch 8/50
43725/43725 ━━━━━━━━━━━━━━━━━━━━ 132s 3ms/step - loss: 2.3344e-06 - val_loss: 4.3612e-06 - learning_rate: 0.0010
Epoch 9/

In [ ]:
print("Evaluate: ", FNN.evaluate(X_test, Y_test))
pred = FNN.predict(X_test)
pred_denormalized = scaler_Y.inverse_transform(pred)
Y_test_denormalized = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))
mse = mean_squared_error(Y_test_denormalized, pred_denormalized)
print("MSE desnormalized", mse)

print("Test Evaluate: ", FNN.evaluate(TestX_normalized, TestY_normalized))
Testpred = FNN.predict(TestX_normalized)
Testpred_denormalized = scaler_Y.inverse_transform(Testpred)
TestY_denormalized = scaler_Y.inverse_transform(TestY_normalized.reshape(-1, 1))

mse = mean_squared_error(TestY_denormalized, Testpred_denormalized)
print("MSE desnormalized", mse)

5466/5466 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 7.5968e-06
Evaluate:  1.3526403563446365e-05
5466/5466 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
MSE desnormalized 0.13703729922274255
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2.9255e-06
Test Evaluate:  2.6767766030388884e-06
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
MSE desnormalized 0.02711685130968437


In [ ]:
FNN.save('BestFNN.keras')

# Results

In [33]:
FNN = load_model('BestFNN.keras')

In [34]:
# Data preprossesing -> X_train, X_test, X_val, Y_train, Y_test, Y_val
# Train - 80%, Test - 10%, Validation - 10%
num_samples = len(X)
train_size = int(0.8 * num_samples)
valid_size = int(0.1 * num_samples)
test_size = int(0.1 * num_samples)

# Split the data into train, validation, and test sets
X_train = X_normalized[:train_size]
Y_train = Y_normalized[:train_size]

X_val = X_normalized[train_size:train_size + valid_size]
Y_val = Y_normalized[train_size:train_size + valid_size]

X_test = X_normalized[train_size + valid_size:]
Y_test = Y_normalized[train_size + valid_size:]


In [35]:
train_predict = FNN.predict(X_train)
val_predict = FNN.predict(X_val)
test_predict = FNN.predict(X_test)

43725/43725 ━━━━━━━━━━━━━━━━━━━━ 8s 176us/step
5466/5466 ━━━━━━━━━━━━━━━━━━━━ 1s 177us/step
5466/5466 ━━━━━━━━━━━━━━━━━━━━ 1s 174us/step


Training

In [36]:
mse = mean_squared_error(train_predict, Y_train)
mae = mean_absolute_error(train_predict, Y_train)
rmse = np.sqrt(mse)
r2 = r2_score(Y_train, train_predict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 6.95677902653285e-07
Mean Absolute Error (MAE): 0.0006641953915547844
RMSE:  0.0008340730799236269
R2:  0.9999753227010855


Validation

In [37]:
mse = mean_squared_error(val_predict, Y_val)
mae = mean_absolute_error(val_predict, Y_val)
rmse = np.sqrt(mse)
r2 = r2_score(Y_val, val_predict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 2.295870122066858e-06
Mean Absolute Error (MAE): 0.0010878103722998746
RMSE:  0.001515212896614485
R2:  0.9999063103233223


Testing

In [38]:

mse = mean_squared_error(test_predict, Y_test)
mae = mean_absolute_error(test_predict, Y_test)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, test_predict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)



Mean Squared Error (MSE): 1.3527303440817743e-05
Mean Absolute Error (MAE): 0.0013949110354950177
RMSE:  0.003677948265108924
R2:  0.9994572053221485


Out of sample

In [39]:
# Normalize TestX
test_model_parameters = TestX.drop(["tau", "stockPrice", "strike"], axis=1)
test_option_properties = TestX.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

normalized_test_model_parameters = pd.DataFrame()
for column in test_model_parameters.columns:
    normalized_test_model_parameters[column] = custom_min_max_normalization(
        test_model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

# Fit the scaler with option_properties from the training data
scaler_option_properties = MinMaxScaler() 
scaler_option_properties.fit(option_properties)

# Transform the option_properties from the test data
normalized_test_option_properties = scaler_option_properties.transform(test_option_properties)
normalized_test_option_properties = pd.DataFrame(normalized_test_option_properties, columns=test_option_properties.columns)

TestX_normalized = pd.concat([normalized_test_model_parameters, normalized_test_option_properties], axis=1).values

# Now the scaler for Y
scaler_Y = MinMaxScaler()
scaler_Y.fit(Y.values.reshape(-1, 1))

# Normalize TestY
TestY_normalized = scaler_Y.transform(TestY.values.reshape(-1, 1))  # Use transform, not fit_transform
TestY_normalized = pd.Series(TestY_normalized.flatten(), name=TestY.name).values

In [41]:
num_iterations = 10

# List to store elapsed times
elapsed_times = []

for _ in range(num_iterations):
    # Start time
    start_time = time.time()

    # Code to measure
    testpredict = FNN.predict(TestX_normalized)

    # End time
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Append elapsed time to list
    elapsed_times.append(elapsed_time)

# Calculate average elapsed time
average_elapsed_time = np.mean(elapsed_times)

print(f"Average execution time over {num_iterations} runs: {average_elapsed_time:.6f} seconds")

1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 175us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 176us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 174us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 191us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 186us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 186us/step
Average execution time over 10 runs: 0.370668 seconds


In [42]:
testpredict = FNN.predict(TestX_normalized)

1325/1325 ━━━━━━━━━━━━━━━━━━━━ 0s 176us/step


In [43]:
mse = mean_squared_error(testpredict, TestY_normalized)
mae = mean_absolute_error(testpredict, TestY_normalized)
rmse = np.sqrt(mse)
r2 = r2_score(TestY_normalized, testpredict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 2.6767739715498964e-06
Mean Absolute Error (MAE): 0.0010227094964302176
RMSE:  0.0016360849524245055
R2:  0.9998820207634256


In [44]:
testpredict_denormalized = scaler_Y.inverse_transform(testpredict.reshape(-1, 1))
TestY_denormalized = scaler_Y.inverse_transform(TestY_normalized.reshape(-1, 1))

mse_denormalized = mean_squared_error(TestY_denormalized, testpredict_denormalized)
mae_denormalized = mean_absolute_error(TestY_denormalized, testpredict_denormalized)
rmse_denormalized = np.sqrt(mse_denormalized)

print("MSE denormalized:", mse_denormalized)
print("MAE denormalized:", mae_denormalized)
print("RMSE denormalized:", rmse_denormalized)
print("R2 denormalized (same as normalized):", r2)

MSE denormalized: 0.02711685098258651
MAE denormalized: 0.1029357152124445
RMSE denormalized: 0.16467194959247464
R2 denormalized (same as normalized): 0.9998820207634256
